In [1]:
import os, sys 
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import numpy as np
import utility as ut
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from scipy import stats
import seaborn as sns
import oneshot as sm
import rfm, skipRFM, deepSkip, localSkip
# import parallel_euler as arch1
import torch
from torch import nn
import l63, glob, count_params
import config as cf
from matplotlib.text import Text
from matplotlib import ticker
from matplotlib.collections import PolyCollection
from scipy.stats import ks_2samp 
import count_params as cp
import wasserstein as ws
import json


dynamical_system = 'L96'
root = f'../data/{dynamical_system}'

In [ ]:
D_r, B = 16384, 1
beta_data = pd.read_csv(f"{root}/config_1_s/LocalDeepSkip_1_4/beta/beta_s.csv")
beta = float(beta_data[(beta_data['D_r']==D_r) & (beta_data['B']==1)]['beta'].iloc[0])
train_kwargs = {"save_folder": f"{root}/config_bnf", "epochs": 15000,\
                "I": 4, "ensemble_size": 32, "model_seed": model_seeds[expr_index]}
data_gen_kwargs = {"dt": 1e-2, "train_size": int(2e5), "train_seed": train_seeds[expr_index], "test_num": 1}
feature_cols = [f"Space_{i}" for i in range(2*train_kwargs["I"]+1)]
drf_kwargs   = {"D_r": D_r, "B": B, "G": 1, "I": 4, "L0": 0.4, "L1": 3.5, "beta": beta}
eval_kwargs = {"vpt_steps": 325, "n_RMSE": 10000, "w2_steps": int(1e1), "vpt_epsilon": 0.5,\
                "Lyapunov_time": 1/2.27, "n_sample_w2": 20000}

In [14]:
# collect bnf data 
n_exprs = 5
folders = [os.path.abspath(f"../../bnf/data/L96/autonomous/D_r-256_B-3_id-{i}") for i in range(n_exprs)]
results = []
for folder in folders:
    with open(f"{folder}/results.json", "r") as file:
        results.append(json.load(file))
avg_res = {}
for k, v in results[0].items():
    avg_res[k] = 0.
    for i in range(n_exprs):
        avg_res[k] += results[i][k]
    avg_res[k] /= n_exprs
for k, v in avg_res.items():
    print(f"{k}: {v}")
print(np.log10(avg_res["model_size"])) 
# recalculate MAE in L2 norm
mae = 0
for folder in folders:
    truth = np.load(f"{folder}/rmse_true_trajectory.npy")
    prediction = np.load(f"{folder}/rmse_trajectory.npy")
    mae += np.linalg.norm(truth-prediction, axis=0, ord=2).mean() 
mae /= len(folders)
print(f"MAE: {mae}")

VPT: 0.9216200000000002
RMSE: 1.6934128761291505
MAE: 7.800250625610351
W2: 37.80690383911133
training_time: 1452.5701784610749
model_size: 2666960.0
experiment_seed: 48.4
model_seed: 42.4
6.426016502038001
MAE: 1.6697739517622299


In [12]:
# collect drf data 
n_exprs = 5
folders = [os.path.abspath(f"../data/L96/config_bnf/expr_{i}") for i in range(n_exprs)]
results = []
for folder in folders:
    with open(f"{folder}/results.json", "r") as file:
        results.append(json.load(file))
avg_res = {}
for k, v in results[0].items():
    avg_res[k] = 0.
    for i in range(n_exprs):
        avg_res[k] += results[i][k]
    avg_res[k] /= n_exprs
for k, v in avg_res.items():
    print(f"{k}: {v}")
print(np.log10(avg_res["model_size"]))

# recalculate MAE in L2 norm
mae = 0
for folder in folders:
    truth = np.load(f"{folder}/rmse_true_trajectory.npy")
    prediction = np.load(f"{folder}/rmse_trajectory.npy")
    mae += np.linalg.norm(truth-prediction, axis=0, ord=2).mean() 
mae /= len(folders)
print(f"MAE: {mae}")

VPT: 4.5354600000000005
RMSE: 1.6873475326485097
MAE: 7.745989144184132
W2: 25.47168960571289
training_time: 5.139635801315308
model_size: 393216.0
experiment_seed: 24.6
model_seed: 52.4
5.594631181007343
MAE: 1.6654620556277884


In [13]:
# collect bnf-time data 
n_exprs = 5
folders = [os.path.abspath(f"../../bnf/data/L96/non-autonomous/D_r-256_B-3_id-{i}") for i in range(n_exprs)]
results = []
for folder in folders:
    try:
        with open(f"{folder}/results.json", "r") as file:
            results.append(json.load(file))
    except:
        pass
avg_res = {}
for k, v in results[0].items():
    avg_res[k] = 0.
    for i in range(n_exprs):
        try:
            avg_res[k] += results[i][k]
        except:
            pass
    avg_res[k] /= n_exprs
for k, v in avg_res.items():
    print(f"{k}: {v}")
print(np.log10(avg_res["model_size"]))
# recalculate MAE in L2 norm
mae = 0
for folder in folders:
    truth = np.load(f"{folder}/rmse_true_trajectory.npy")
    prediction = np.load(f"{folder}/rmse_trajectory.npy")
    mae += np.linalg.norm(truth-prediction, axis=0, ord=2).mean() 
mae /= len(folders)
print(f"MAE: {mae}")

VPT: 0.41768
RMSE: 1.7177155494689942
MAE: 7.973823738098145
W2: 39.18142852783203
training_time: 1590.834502029419
model_size: 2748912.0
experiment_seed: 50.0
model_seed: 57.2
6.439160837142075
MAE: 1.693235468846808


In [28]:
393216*16

6291456

In [27]:
2666960/16

166685.0